[View in Colaboratory](https://colab.research.google.com/github/gauravbansal98/Save-and-Restore-a-model-in-tensorflow/blob/master/Save_and_restore_and_retrain_a_model.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
!mkdir -p drive

!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import os
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("drive/Uploaded on github/mnist dataset using rnn", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting drive/Uploaded on github/mnist dataset using rnn/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting drive/Uploaded on github/mnist dataset using rnn/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting drive/Uploaded on github/mnist dataset using rnn/t10k-images-idx3-ubyte.gz
Extracting drive/Uploaded on github/mnist dataset using rnn/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
from tensorflow.python.ops import rnn, rnn_cell
tf.reset_default_graph()


In [0]:
rnn_size = 128
no_of_nodes_in_output_layer = 10
chunk_size = 56
no_of_chunk = 14
batch_size = 128
hm_epochs = 1

In [0]:
x = tf.placeholder(dtype = tf.float32, shape = [None, no_of_chunk, chunk_size], name = 'input_labels')
y = tf.placeholder(dtype = tf.float32, shape = [None, None], name = 'output_labels')

In [0]:
weights = { 'weights' : tf.Variable(tf.random_normal([rnn_size, no_of_nodes_in_output_layer])),
           'biases' : tf.Variable(tf.random_normal([1, no_of_nodes_in_output_layer])) }

In [0]:
def recurrent_neural_network(x):
  with tf.name_scope('Neural_Network'):
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, no_of_chunk, 0)
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size, name = 'LSTM_CELL')
    outputs, state = tf.nn.static_rnn(cell, x, dtype = tf.float32)

    output = tf.add(tf.matmul(outputs[-1], weights['weights']), weights['biases'], name = 'last_layer_output')
    return output

In [48]:
os.chdir('drive/Uploaded on github/Save and Restore a model in tensorflow')

prediction = recurrent_neural_network(x)

y_pred = tf.nn.softmax(prediction, name = "y_pred")

saver = tf.train.Saver(tf.trainable_variables())

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y), name = "cost")

optimizer = tf.train.AdamOptimizer().minimize(cost)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for i in range(hm_epochs):
    epoch_loss = 0
    for j in range(int(mnist.train.num_examples/batch_size)):
      epoch_x, epoch_y = mnist.train.next_batch(batch_size)
      epoch_x = epoch_x.reshape((batch_size,no_of_chunk,chunk_size))
      o, c = sess.run([optimizer, cost], feed_dict = {x : epoch_x, y : epoch_y})
      epoch_loss += c
    print('epoch' , i, 'completed out of ', hm_epochs, 'loss ', epoch_loss)

  saver.save(sess, './saved_model')
  
  correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

  
  accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

  print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape(-1,no_of_chunk,chunk_size), y:mnist.test.labels})) 

epoch 0 completed out of  1 loss  245.44157840311527
Accuracy: 0.9453


In [29]:
hm_epochs = 2

batch_size = 128

tf.reset_default_graph()

sess = tf.Session()

saver = tf.train.import_meta_graph('saved_model.meta')

print("meta graph loaded")

saver.restore(sess, tf.train.latest_checkpoint('./'))

graph = tf.get_default_graph()

x = graph.get_tensor_by_name("input_labels:0")

y = graph.get_tensor_by_name("output_labels:0")

last_layer_output = graph.get_tensor_by_name("Neural_Network/last_layer_output:0")

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = last_layer_output, labels = y), name = "cost")

optimizer = tf.train.AdamOptimizer(name = "sss")

minimize = optimizer.minimize(cost) 

sess.run(tf.variables_initializer(optimizer.variables()))

var = tf.trainable_variables()

var_before = sess.run(var)

for i in range(2):
  epoch_loss = 0
  for j in range(int(mnist.train.num_examples/batch_size)):
    epoch_x, epoch_y = mnist.train.next_batch(batch_size)
    epoch_x = epoch_x.reshape((batch_size,no_of_chunk,chunk_size))
    o, c = sess.run([minimize, cost], feed_dict = {x : epoch_x, y : epoch_y})
    epoch_loss += c
  print('epoch' , i, 'completed out of ', hm_epochs, 'loss ', epoch_loss)

new_saver = tf.train.Saver(tf.trainable_variables())
new_saver.save(sess, './saved_model')

var = tf.trainable_variables()

var_after = sess.run(var)

correct = tf.equal(tf.argmax(last_layer_output, 1), tf.argmax(y, 1))
  
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

print('Accuracy:',sess.run(accuracy, feed_dict = {x:mnist.test.images.reshape(-1,no_of_chunk,chunk_size), y:mnist.test.labels})) 

sess.close()

meta graph loaded
INFO:tensorflow:Restoring parameters from ./saved_model
epoch 0 completed out of  2 loss  26.562956606037915
epoch 1 completed out of  2 loss  19.843446370447055
Accuracy: 0.9823
